In [1]:
# loading model
import pandas as pd
import pickle
from tqdm import tqdm
import random
import numpy as np
import xgboost as xgb
pd.set_option('display.max_columns', 50)

logreg_model = pickle.load(open("logreg_model.pkl", "rb"))
xgb_model = pickle.load(open("xgb_model.pkl", "rb"))
gnb_model = pickle.load(open("gnb_model.pkl", "rb"))

sample_submission_final = pd.read_csv('data/SampleSubmission2023.csv')
sample_submission_warmup = pd.read_csv('data/SampleSubmissionWarmup.csv')

In [2]:
reg_data = pd.read_csv('computed_data/reg_avg_data.csv')
reg_data = reg_data[reg_data['DayNum']<134]
reg_data

,Season,DayNum,A,B,Gender,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
0,2003,12,1186,1457,1,-26.000000,-6.000000,-3.152709,-6.060606,15.749526,-0.495614,-0.608889,-0.166005,8.350000,0.000000,-22.727273,-31.851852,-0.859649,-0.362319,0.084367,-8.700000,1
1,2003,12,1458,1296,1,26.000000,6.000000,0.000000,22.727273,31.851852,0.859649,0.362319,-0.084367,8.700000,3.152709,6.060606,-15.749526,0.495614,0.608889,0.166005,-8.350000,1
2,2003,14,1135,1125,1,-1.000000,-28.000000,3.194103,-11.899314,-4.859335,-0.440171,-1.615385,0.415305,-3.850000,-19.680851,7.017544,-19.230769,-0.176471,-0.294118,-0.340612,2.625000,0
3,2003,14,1236,1156,1,-18.000000,-5.000000,8.756137,-18.787879,15.873016,-0.162745,0.021739,0.238555,0.750000,8.260325,8.333333,-37.606838,-0.303529,0.075556,0.009551,-0.700000,0
4,2003,14,1161,1194,1,18.000000,5.000000,-8.260325,-8.333333,37.606838,0.303529,-0.075556,-0.009551,0.700000,-8.756137,18.787879,-15.873016,0.162745,-0.021739,-0.238555,-0.750000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173012,2023,132,3125,3179,0,5.727273,13.466667,11.349490,3.707446,-4.497852,0.372995,-0.001240,0.023285,-6.010530,13.668956,3.453100,0.380525,0.255284,0.506572,-0.026285,5.726061,0
173013,2023,132,3221,3131,0,6.800000,11.806452,7.391650,2.655703,-0.317925,0.122910,0.177077,0.006133,-2.274839,9.277848,8.063700,7.929181,0.390162,0.136066,-0.000226,2.505860,1
173014,2023,132,3235,3400,0,11.433333,16.606061,10.866210,0.123848,10.716760,0.708295,0.232672,-0.139222,1.174167,6.388167,4.035456,0.374459,-0.017555,0.246038,0.128870,-1.133333,1
173015,2023,132,3284,3406,0,-0.064516,5.225806,7.048193,0.896605,-6.590451,-0.008355,0.201963,-0.022104,-3.431452,0.010459,-1.511386,2.680660,-0.148304,-0.069711,0.003560,2.950806,1


In [3]:
reg_data.iloc[:,4:-1].columns

Index(['Gender', 'PM_A', 'PM_B', 'margin_FGP2_A', 'margin_FGP3_A',
       'margin_FTP_A', 'margin_AST_TO_A', 'margin_STBL_PF_A',
       'margin_REB_DIFF_A', 'margin_POSS_A', 'margin_FGP2_B', 'margin_FGP3_B',
       'margin_FTP_B', 'margin_AST_TO_B', 'margin_STBL_PF_B',
       'margin_REB_DIFF_B', 'margin_POSS_B'],
      dtype='object')

In [4]:
def get_yearly_team(data, year, team_id):
    data = data[(data['Season']==year)&((data['A'] == team_id)|(data['B'] == team_id))].tail(1)
    #print("Actual Data")
    #display(data)
    list_data = []
    if (team_id == data.iloc[0]['A']):
        list_data.append(data.iloc[0]['PM_A'])
        list_data.append(data.iloc[0]['margin_FGP2_A'])
        list_data.append(data.iloc[0]['margin_FGP3_A'])
        list_data.append(data.iloc[0]['margin_FTP_A'])
        list_data.append(data.iloc[0]['margin_AST_TO_A'])
        list_data.append(data.iloc[0]['margin_STBL_PF_A'])
        list_data.append(data.iloc[0]['margin_REB_DIFF_A'])
        list_data.append(data.iloc[0]['margin_POSS_A'])
    else:
        list_data.append(data.iloc[0]['PM_B'])
        list_data.append(data.iloc[0]['margin_FGP2_B'])
        list_data.append(data.iloc[0]['margin_FGP3_B'])
        list_data.append(data.iloc[0]['margin_FTP_B'])
        list_data.append(data.iloc[0]['margin_AST_TO_B'])
        list_data.append(data.iloc[0]['margin_STBL_PF_B'])
        list_data.append(data.iloc[0]['margin_REB_DIFF_B'])
        list_data.append(data.iloc[0]['margin_POSS_B'])
    return list_data

In [5]:
%%time
print("Getting teams.")
#mTourneyseeds = pd.read_csv('data/MNCAATourneySeeds.csv')
#wTourneyseeds = pd.read_csv('data/WNCAATourneySeeds.csv')
#tourney_teams = []
#submission_data_logreg = []
#submission_data_xgb = []
submission_data_gnb = []
#for index, row in mTourneyseeds.iterrows():
#    tourney_teams.append(row['TeamID'])
sample_submission = sample_submission_warmup
#np.sort(np.unique(tourney_teams))
c = 0
for row in tqdm(sample_submission.itertuples()):
    year = int(row.ID.split('_')[0])
    team_1 = int(row.ID.split('_')[1])
    team_2 = int(row.ID.split('_')[2])
    gender = -1
    label = str(year) + '_' + str(team_1) + '_' + str(team_2)
    #print(team_1, team_2)
    if (team_1 > 3000):
        gender = 0
    else:
        gender = 1
    try:
        list_data = []
        *stats_A, = get_yearly_team(reg_data, year, team_1)
        *stats_B, = get_yearly_team(reg_data, year, team_2)
        list_data.append([gender, *stats_A, *stats_B])
        data = pd.DataFrame(list_data, columns = ['Gender', 'PM_A', 'margin_FGP2_A', 'margin_FGP3_A',
           'margin_FTP_A', 'margin_AST_TO_A', 'margin_STBL_PF_A',
           'margin_REB_DIFF_A', 'margin_POSS_A', 'PM_B', 'margin_FGP2_B', 'margin_FGP3_B',
           'margin_FTP_B', 'margin_AST_TO_B', 'margin_STBL_PF_B',
           'margin_REB_DIFF_B', 'margin_POSS_B'])
        data = data.reindex(columns = ['Gender', 'PM_A', 'PM_B', 'margin_FGP2_A', 'margin_FGP3_A',
           'margin_FTP_A', 'margin_AST_TO_A', 'margin_STBL_PF_A',
           'margin_REB_DIFF_A', 'margin_POSS_A', 'margin_FGP2_B', 'margin_FGP3_B',
           'margin_FTP_B', 'margin_AST_TO_B', 'margin_STBL_PF_B',
           'margin_REB_DIFF_B', 'margin_POSS_B'])
        #print("Final")
        #display(data)
        #prediction_xgb = xgb_model.predict(xgb.DMatrix(data))[0]
        #prediction_logreg = logreg_model.predict_proba(data)[:,1][0]
        prediction_gnb = gnb_model.predict_proba(data)[:,1][0]
        #submission_data_logreg.append([label, prediction_logreg])
        #submission_data_xgb.append([label, prediction_xgb])
        submission_data_gnb.append([label, prediction_gnb])
    except:
        c+=1
        #submission_data_logreg.append([label, 0.5])
        #submission_data_xgb.append([label, 0.5])
        submission_data_gnb.append([label, prediction_gnb])
print(c)

Getting teams.


614319it [32:44, 312.75it/s]

687
CPU times: total: 32min 46s
Wall time: 32min 44s


In [15]:
reg_data[(reg_data['Season']==2021)&((reg_data['A'] == 3101)|(reg_data['B'] == 3101))].tail(1)

,Season,DayNum,A,B,Gender,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
152239,2021,129,3311,3101,0,-5.681818,-1.055556,-2.57575,0.011606,2.93718,-0.664456,0.08575,0.008041,4.104924,4.393317,2.436579,5.807148,-0.013957,-0.320443,-0.047631,-5.012626,1


In [17]:
reg_data[(reg_data['Season']==2021)&((reg_data['A'] == 3167)|(reg_data['B'] == 3167))].tail(1)

,Season,DayNum,A,B,Gender,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
152192,2021,128,3167,3218,0,-0.705882,-1.538462,3.127253,0.68219,-2.253412,-0.278608,0.427669,0.054655,-1.849774,0.650693,-2.45378,0.386431,0.005327,-0.552881,-0.055643,2.578507,0


In [21]:
submission_data_gnb

[['2023_1101_1102', 0.3911902140005419],
 ['2023_1101_1103', 0.3053604183684928],
 ['2023_1101_1104', 0.2497701874387968],
 ['2023_1101_1105', 0.5208202409327847],
 ['2023_1101_1106', 0.579195787010333],
 ['2023_1101_1107', 0.597162764444718],
 ['2023_1101_1108', 0.496128009965934],
 ['2023_1101_1110', 0.4400721986706994],
 ['2023_1101_1111', 0.4612612162111915],
 ['2023_1101_1112', 0.21973609314920425],
 ['2023_1101_1113', 0.3982282498566343],
 ['2023_1101_1114', 0.5356879268146226],
 ['2023_1101_1115', 0.5819141916159052],
 ['2023_1101_1116', 0.3323993090610561],
 ['2023_1101_1117', 0.5401714430746406],
 ['2023_1101_1119', 0.4341653489729333],
 ['2023_1101_1120', 0.32567630477928233],
 ['2023_1101_1122', 0.5822961790204658],
 ['2023_1101_1123', 0.4001381650073964],
 ['2023_1101_1124', 0.30304541024961723],
 ['2023_1101_1125', 0.33033971735651557],
 ['2023_1101_1126', 0.5539330408758582],
 ['2023_1101_1127', 0.5229982555293631],
 ['2023_1101_1129', 0.3309183095287882],
 ['2023_1101_11

In [7]:
submission_data_xgb

[['2023_1101_1102', 0.42036194],
 ['2023_1101_1103', 0.36747804],
 ['2023_1101_1104', 0.28763413],
 ['2023_1101_1105', 0.5284094],
 ['2023_1101_1106', 0.56648606],
 ['2023_1101_1107', 0.5754033],
 ['2023_1101_1108', 0.5165108],
 ['2023_1101_1110', 0.46327016],
 ['2023_1101_1111', 0.4659927],
 ['2023_1101_1112', 0.29221037],
 ['2023_1101_1113', 0.395744],
 ['2023_1101_1114', 0.5394541],
 ['2023_1101_1115', 0.5754033],
 ['2023_1101_1116', 0.32332715],
 ['2023_1101_1117', 0.5246473],
 ['2023_1101_1119', 0.4426818],
 ['2023_1101_1120', 0.36747804],
 ['2023_1101_1122', 0.56648606],
 ['2023_1101_1123', 0.4315222],
 ['2023_1101_1124', 0.3020623],
 ['2023_1101_1125', 0.3691051],
 ['2023_1101_1126', 0.5623652],
 ['2023_1101_1127', 0.54958475],
 ['2023_1101_1129', 0.33757868],
 ['2023_1101_1130', 0.52664137],
 ['2023_1101_1131', 0.5029859],
 ['2023_1101_1132', 0.52664137],
 ['2023_1101_1133', 0.3103072],
 ['2023_1101_1135', 0.44162682],
 ['2023_1101_1136', 0.432024],
 ['2023_1101_1137', 0.513946

In [6]:
#submission_data_logreg_df = pd.DataFrame(submission_data_logreg, columns = ['ID','Pred'])
#submission_data_xgb_df = pd.DataFrame(submission_data_xgb, columns = ['ID','Pred'])
#submission_data_logreg_df.to_csv('computed_data/submission_logreg.csv',index=False)
#submission_data_xgb_df.to_csv('computed_data/submission_xgb.csv',index=False)
submission_data_gnb_df = pd.DataFrame(submission_data_gnb, columns = ['ID','Pred'])
submission_data_gnb_df.to_csv('computed_data/submission_gnb.csv',index=False)